In [ ]:
import pandas as pd
df = pd.read_sql(session.query(ForeclosureCaseSchema).statement, engine)
df.info()

In [ ]:
    @model_validator(mode='before')
    def convert_verkehrswert(cls, values):
        verkehrswert_str = values.get('verkehrswert_in_€')
        
        if verkehrswert_str and isinstance(verkehrswert_str, str):
            # Handle cases where multiple values are separated by ":"
            object_values = verkehrswert_str.split(':')
            total_value = 0.0
            
            for obj_value in object_values:
                # Only take strings that contain digits and end with "€"
                match = re.search(
                    r'(\d+[.,\d]*)\s*€' if len(object_values) > 1 else r'(\d+[.,\d]*)',
                    obj_value
                )
                if match:
                    value_str = match.group(1)
                    # Remove any non-numeric characters except "," and "."
                    value_str = re.sub(r'[^\d,.]', '', value_str).strip()
                    # Handle the German number format: 650.000,46 -> 650000.46
                    value_str = value_str.replace('.', '').replace(',', '.')
                    
                    try:
                        total_value += float(value_str)
                    except ValueError:
                        # Skip invalid float conversions
                        pass

            values['verkehrswert_in_€'] = total_value
        
        return values